In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
data_path = PATH['model_test_pred']
csv_path = PATH['annotations_test']
df_anno = pd.read_csv(csv_path + "bbfinal.csv")

In [3]:
patients = [x for x in os.listdir(data_path) if 'orig.npy' in x]

In [4]:
df_anno["file"] = df_anno["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
df_anno = df_anno.dropna()

In [5]:
empty = pandas.DataFrame({'seriesuid':[],'coordX':[],'coordY':[],'coordZ':[],"probability":[],'labels':[]})
width = height = 512
for img_file in tqdm(sorted(patients)):
    mini_df_anno = df_anno[df_anno["file"]==img_file] 
    if mini_df_anno.shape[0]>0:
        patient_id = img_file[:-9]
        img_array = np.load(data_path + img_file)

        pos_annos = pd.read_csv(data_path + img_file[:-9] + '_annos_pos.csv')
        origin = np.array([pos_annos.loc[0]['origin_x'],pos_annos.loc[0]['origin_y'],pos_annos.loc[0]['origin_z']]) 
        spacing = np.array([pos_annos.loc[0]['spacing_x'],pos_annos.loc[0]['spacing_y'],pos_annos.loc[0]['spacing_z']]) 
        
        for node_idx, cur_row in mini_df_anno.iterrows():             
            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            prob = cur_row["probability"]
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint(np.absolute(center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)

            img = img_array[int(v_center[2])]
            seg_img, overlap = helpers.get_segmented_lungs(img.copy())
            img = img*overlap
            mask = make_mask(center, 4, int(v_center[2])*spacing[2]+origin[2],
                                 width, height, spacing, origin)
            labels = len(np.unique(skimage.measure.label(mask*overlap)))
            insertrow0 = pd.DataFrame([[patient_id,node_x,node_y,node_z,prob,labels]],columns = ['seriesuid','coordX','coordY','coordZ',"probability",'labels'])
            
            #cv2.imwrite(pic_path + str(patient_id)+'_'+str(node_idx)+'_'+str(i)+ '_i.png',img*255)
            #cv2.imwrite(pic_path + str(patient_id)+'_'+str(node_idx)+'_'+str(i)+ '_m.png',mask*255)
            #cv2.imwrite(pic_path + str(patient_id)+'_'+str(node_idx)+'_'+str(i)+ '_o.png',overlap*255)        
            
            empty = empty.append(insertrow0,ignore_index=True)
empty = empty[['seriesuid','coordX','coordY','coordZ',"probability",'labels']]

100%|██████████| 200/200 [30:47<00:00,  9.75s/it]


In [6]:
empty.to_csv(csv_path + "bbfinal_filt.csv", index=False)

In [7]:
empty

,seriesuid,coordX,coordY,coordZ,probability,labels
0,LKDS-01004,-30.292711,-134.998786,12.162719,0.017,2.0
1,LKDS-01004,-79.896122,5.639107,20.399999,0.005,1.0
2,LKDS-01004,-183.468330,-168.430704,42.185278,0.005,1.0
3,LKDS-01004,42.481598,-140.302859,62.933097,0.005,5.0
4,LKDS-01004,-64.684913,-129.913196,53.464393,0.005,5.0
5,LKDS-01004,-40.828513,-153.831798,55.230468,0.005,4.0
6,LKDS-01004,197.393584,-148.941298,56.999999,0.005,1.0
7,LKDS-01004,49.014468,-163.072749,61.019757,0.005,6.0
8,LKDS-01004,-37.717700,-169.590000,66.523912,0.005,3.0
9,LKDS-01004,-67.026760,-143.632532,60.752082,0.005,7.0
